In [120]:
import pandas
import pysdql

path = r'T:\UG4-Proj\datasets'
answer_path = r'T:\UG4-Proj\datasets'

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)
partsupp = pandas.read_table(rf'{path}\partsupp.tbl', sep='|', index_col=False, header=None, names=pysdql.PARTSUPP_COLS)
region = pandas.read_table(rf'{path}\region.tbl', sep='|', index_col=False, header=None, names=pysdql.REGION_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)

# r = r.merge(orders, how='inner', left_on='o_orderkey', right_on='l_orderkey')

In [121]:
# TPCH - 1
'''
l_shipdate <= date '1998-12-01' - interval ':1' day (3)
'''
var1 = '1998-09-05'
r = lineitem[lineitem['l_shipdate'] <= var1]

r['disc_price'] = r['l_extendedprice'] * (1 - r['l_discount'])
r['charge'] = r['l_extendedprice'] * (1 - r['l_discount']) * (1 + r['l_tax'])

r = r.groupby(['l_returnflag', 'l_linestatus'], as_index=False).agg(sum_qty=('l_quantity', 'sum'),
                                                    sum_base_price=('l_extendedprice', 'sum'),
                                                    sum_disc_price=('disc_price', 'sum'),
                                                    sum_charge=('charge', 'sum'),
                                                    avg_qty=('l_quantity', 'mean'),
                                                    avg_price=('l_extendedprice', 'mean'),
                                                    avg_disc=('l_discount', 'mean'),
                                                    count_order=('l_returnflag', 'count')
                                                    )
r

C:\Users\Y\AppData\Local\Temp\ipykernel_19592\1025114352.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['disc_price'] = r['l_extendedprice'] * (1 - r['l_discount'])
C:\Users\Y\AppData\Local\Temp\ipykernel_19592\1025114352.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['charge'] = r['l_extendedprice'] * (1 - r['l_discount']) * (1 + r['l_tax'])


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,37474,37569624.64,3.567619e+07,3.710142e+07,25.354533,25419.231827,0.050866,1478
1,N,F,1041,1041301.07,9.990609e+05,1.036451e+06,27.394737,27402.659737,0.042895,38
2,N,O,75354,75571811.04,7.182923e+07,7.467892e+07,25.561058,25634.942687,0.049715,2948
3,R,F,36511,36570841.24,3.473847e+07,3.616906e+07,25.059025,25100.096939,0.050027,1457


In [150]:
# TPCH - 2
var1 = 14
var2 = 'BRASS'
var3 = 'EUROPE'

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)
partsupp = pandas.read_table(rf'{path}\partsupp.tbl', sep='|', index_col=False, header=None, names=pysdql.PARTSUPP_COLS)
region = pandas.read_table(rf'{path}\region.tbl', sep='|', index_col=False, header=None, names=pysdql.REGION_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)

sub_r = region[(region['r_name'] == var3)]

r1 = partsupp.merge(supplier, left_on='ps_suppkey', right_on='s_suppkey')\
    .merge(nation, left_on='s_nationkey', right_on='n_nationkey')\
    .merge(sub_r, left_on='n_regionkey', right_on='r_regionkey')
r1 = r1.groupby(['ps_partkey', 'ps_suppkey'], as_index=False).agg(min_supplycost=('ps_supplycost', 'min'))

r1['min_partkey'] = r1['ps_partkey']
r1['min_suppkey'] = r1['ps_suppkey']

sub_p = part[part['p_size'] == var1]
sub_p = sub_p[sub_p['p_type'].str.endswith(var2)]

r2 = sub_p.merge(partsupp, how='inner', left_on='p_partkey', right_on='ps_partkey')\
    .merge(supplier, how='inner', left_on='ps_suppkey', right_on='s_suppkey')\
    .merge(nation, how='inner', left_on='s_nationkey', right_on='n_nationkey')\
    .merge(sub_r, how='inner', left_on='n_regionkey', right_on='r_regionkey')

r = r1.merge(r2,
             left_on=['min_partkey', 'min_suppkey', 'min_supplycost'],
             right_on=['ps_partkey', 'ps_suppkey', 'ps_supplycost'])

r[['s_acctbal', 's_name', 'n_name', 'p_partkey', 'p_mfgr', 's_address', 's_phone', 's_comment']]

,s_acctbal,s_name,n_name,p_partkey,p_mfgr,s_address,s_phone,s_comment
0,6820.35,Supplier#000000007,UNITED KINGDOM,4,Manufacturer#3,"s,4TicNGB4uO6PaSqNBUq",33-990-965-2201,s unwind silently furiously regular courts. fi...


In [123]:
# TPCH - 3

import pandas

var1 = 'BUILDING'
var2 = '1995-03-22'

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)

r = customer.merge(orders, how='inner', left_on='c_custkey', right_on='o_custkey')
r = r.merge(lineitem, how='inner', left_on='o_orderkey', right_on='l_orderkey')

r = r[(r['c_mktsegment'] == var1)
      & (r['o_orderdate'] < var2)
      & (r['l_shipdate'] > var2)]

r['value'] = r['l_extendedprice'] * (1 - r['l_discount'])

r.groupby(['l_orderkey', 'o_orderdate', 'o_shippriority'], as_index=False) \
    .agg(revenue=('value', 'sum'))

,l_orderkey,o_orderdate,o_shippriority,revenue
0,742,1994-12-23,0,43728.0480
1,775,1995-03-18,0,55713.6063
2,998,1994-11-26,0,6811.4340
3,1637,1995-02-08,0,87438.8481
4,2883,1995-01-23,0,36666.9612
5,3430,1994-12-12,0,4726.6775
6,3492,1994-11-24,0,43716.0724
7,5191,1994-12-11,0,49378.3094


In [183]:
# TPCH - 4

import pandas
import numpy

var1 = '1996-05-01'
var2 = '1996-08-01'  # var1 + 3 month

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)

# sub_l = lineitem[]
sub_o = orders[(orders['o_orderdate'] >= var1) & (orders['o_orderdate'] < var2)]

r = sub_o.merge(lineitem, left_on='o_orderkey', right_on='l_orderkey')

r['exists'] = numpy.select(
[
    (r['l_commitdate'] < r['l_receiptdate'])
],
[
    1
],
default=0)

r = r.groupby(['o_orderpriority', 'o_orderkey'], as_index=False).agg(exists=('exists', 'sum'))
r = r[r['exists'] > 0]

r.groupby(['o_orderpriority'], as_index=False).agg(order_count=('o_orderkey', 'count'))

,o_orderpriority,order_count
0,1-URGENT,10
1,2-HIGH,6
2,3-MEDIUM,9
3,4-NOT SPECIFIED,12
4,5-LOW,4


In [125]:
# TPCH - 5
import pandas

var1 = 'MIDDLE EAST'
var2 = '1995-01-01'
var3 = '1996-01-01'  # var2 + 1 year

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
region = pandas.read_table(rf'{path}\region.tbl', sep='|', index_col=False, header=None, names=pysdql.REGION_COLS)

r = customer.merge(orders, how='inner', left_on='c_custkey', right_on='o_custkey')
r = r.merge(lineitem, how='inner', left_on='o_orderkey', right_on='l_orderkey')
r = r.merge(supplier, how='inner', left_on='l_suppkey', right_on='s_suppkey')
r = r.merge(nation, how='inner', left_on='s_nationkey', right_on='n_nationkey')
r = r.merge(region, how='inner', left_on='n_regionkey', right_on='r_regionkey')

r = r[r['c_nationkey'] == r['s_nationkey']]

r = r[(r['r_name'] == var1)
      & (r['o_orderdate'] >= var2)
      & (r['o_orderdate'] < var3)]

r['value'] = r['l_extendedprice'] * (1 - r['l_discount'])

r.groupby(['n_name'], as_index=False).agg(revenue=('value', 'sum'))

,n_name,revenue
0,IRAN,64854.8802
1,IRAQ,28957.4409


In [126]:
# TPCH - 6

import pandas
import pysdql

var1 = '1997-01-01'
var2 = '1998-01-01'  # var1 + 1 year
discount = 0.06
var3 = round(discount - 0.01, 2)
var4 = round(discount + 0.01, 2)
var5 = 24

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)



r = lineitem[(lineitem.l_shipdate >= var1) & (lineitem.l_shipdate < var2)
             & (lineitem.l_discount > var3) & (lineitem.l_discount < var4)
             & (lineitem.l_quantity < var5)]

r['value'] = r['l_extendedprice'] * r['l_discount']

r.agg(revenue=('value', 'sum'))


C:\Users\Y\AppData\Local\Temp\ipykernel_19592\1543105238.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['value'] = r['l_extendedprice'] * r['l_discount']


,value
revenue,23106.0642


In [162]:
# TPCH - 7

import pandas

var1 = 'PERU'
var2 = 'MOROCCO'

supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
n1_cols = ['n1_nationkey', 'n1_name', 'n1_regionkey', 'n1_comment']
n2_cols = ['n2_nationkey', 'n2_name', 'n2_regionkey', 'n2_comment']
n1 = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=n1_cols)
n2 = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=n2_cols)

r = n1.merge(n2, on=((n1['n1_name'] == var1) & (n2['n2_name'] == var2))
                    | ((n1['n1_name'] == var2) & (n2['n2_name'] == var1)))

sub_n = n1.merge(n2, how='cross')
r = sub_n[(sub_n['n1_name'] == var1) & (sub_n['n2_name'] == var2)
          | ((sub_n['n1_name'] == var2) & (sub_n['n2_name'] == var1))]

r = r.merge(customer, left_on='n2_nationkey', right_on='c_nationkey')
r = r.merge(supplier, left_on='n1_nationkey', right_on='s_nationkey')
r = r.merge(orders, left_on='c_custkey', right_on='o_custkey')

sub_l = lineitem[(lineitem['l_shipdate'] >= '1995-01-01') & (lineitem['l_shipdate'] <= '1996-12-31')]
r = r.merge(sub_l, left_on=['o_orderkey', 's_suppkey'], right_on=['l_orderkey', 'l_suppkey'])

r['supp_nation'] = r['n1_name']
r['cust_nation'] = r['n2_name']
r['volume'] = r['l_extendedprice'] * (1 - r['l_discount'])
r['l_year'] = pandas.DatetimeIndex(r['l_shipdate']).year

r.groupby(['supp_nation', 'cust_nation', 'l_year'], as_index=False).agg(revenue=('volume', 'sum'))

,supp_nation,cust_nation,l_year,revenue
0,MOROCCO,PERU,1995,228013.9886
1,MOROCCO,PERU,1996,180769.5690
2,PERU,MOROCCO,1995,306687.8357
3,PERU,MOROCCO,1996,203568.8640


In [165]:
# TPCH - 8

import pandas
import numpy
import pysdql

var1 = 'MOROCCO'
var2 = 'MIDDLE EAST'
var3 = 'SMALL ANODIZED COPPER'

part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
region = pandas.read_table(rf'{path}\region.tbl', sep='|', index_col=False, header=None, names=pysdql.REGION_COLS)

n1_cols = ['n1_nationkey', 'n1_name', 'n1_regionkey', 'n1_comment']
n2_cols = ['n2_nationkey', 'n2_name', 'n2_regionkey', 'n2_comment']
n1 = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=n1_cols)
n2 = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=n2_cols)

sub_r = region[(region['r_name'] == var2)]
sub_o = orders[(orders['o_orderdate'] >= '1995-01-01') & (orders['o_orderdate'] <= '1996-12-31')]

r1 = n1.merge(sub_r, left_on='n1_regionkey', right_on='r_regionkey')

r2 = part.merge(lineitem, left_on='p_partkey', right_on='l_partkey')
r2 = r2.merge(orders, left_on='l_orderkey', right_on='o_orderkey')
r2 = r2.merge(customer, left_on='o_custkey', right_on='c_custkey')

r = r1.merge(r2, left_on='n1_nationkey', right_on='c_nationkey')
r = r.merge(supplier, left_on='l_suppkey', right_on='s_suppkey')
r = r.merge(n2, left_on='s_nationkey', right_on='n2_nationkey')

r['o_year'] = pandas.DatetimeIndex(r['o_orderdate']).year
r['volume'] = r['l_extendedprice'] * (1 - r['l_discount'])
r['nation'] = r['n2_name']

r = r[['o_year', 'volume', 'nation']]

r['value1'] = numpy.where(r['nation'] == var1, r['volume'], 0)

r = r.groupby(['o_year'], as_index=False).agg(value2=('value1', 'sum'), value3=('volume', 'sum'))

r['revenue'] = r['value2'] / r['value3']

r = r.drop(columns=['value2', 'value3'])

r

,o_year,revenue
0,1992,0.083783
1,1993,0.115800
2,1994,0.128000
3,1995,0.082131
4,1996,0.125664
5,1997,0.108439
6,1998,0.144390


In [153]:
# TPCH - 9
var1 = 'cornflower'

part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
partsupp = pandas.read_table(rf'{path}\partsupp.tbl', sep='|', index_col=False, header=None, names=pysdql.PARTSUPP_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)

sub_p = part[part['p_name'].str.startswith(var1)]

r1 = sub_p.merge(partsupp, how='inner', left_on='p_partkey', right_on='ps_partkey')
r2 = supplier.merge(nation, how='inner', left_on='s_nationkey', right_on='n_nationkey')
r2 = r2.merge(r1, how='inner', left_on='s_suppkey', right_on='ps_suppkey')
r = r2.merge(lineitem, how='inner', left_on='s_suppkey', right_on='l_suppkey')
r = r[r['ps_suppkey'] == r['l_suppkey']]
r = r.merge(orders, how='inner', left_on='l_orderkey', right_on='o_orderkey')

r['nation'] = r['n_name']
r['o_year'] = pandas.DatetimeIndex(r['o_orderdate']).year
r['amount'] = r['l_extendedprice'] * (1 - r['l_discount']) - r['ps_supplycost'] * r['l_quantity']

r = r[['nation', 'o_year', 'amount']]

r.groupby(['nation', 'o_year'], as_index=False).agg(sum_profit=('amount', 'sum'))

,nation,o_year,sum_profit
0,ARGENTINA,1992,1.826905e+06
1,ARGENTINA,1993,1.708000e+06
2,ARGENTINA,1994,1.682303e+06
3,ARGENTINA,1995,1.642867e+06
4,ARGENTINA,1996,1.600254e+06
5,ARGENTINA,1997,1.551643e+06
6,ARGENTINA,1998,1.034894e+06
7,ETHIOPIA,1992,6.783066e+04
8,ETHIOPIA,1993,1.310032e+05
9,ETHIOPIA,1994,1.249977e+05


In [130]:
# TPCH - 10
var1 = '1993-08-01'
var2 = '1993-11-01'  # var1 + 3 month

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)

part_o = orders[(orders['o_orderdate'] >= var1) & (orders['o_orderdate'] < var2)]

r = customer.merge(part_o, how='inner', left_on='c_custkey', right_on='o_custkey')
r = r.merge(nation, how='inner', left_on='c_nationkey', right_on='n_nationkey')

part_l = lineitem[lineitem['l_returnflag'] == 'R']
r = r.merge(part_l, how='inner', left_on='o_custkey', right_on='l_orderkey')

r['value'] = r['l_extendedprice'] * (1 - r['l_discount'])

r.groupby(['c_custkey', 'c_name', 'c_acctbal', 'c_phone', 'n_name', 'c_address', 'c_comment'], as_index=False).agg(revenue=('value', 'sum'))

,c_custkey,c_name,c_acctbal,c_phone,n_name,c_address,c_comment,revenue
0,64,Customer#000000064,-646.64,13-558-731-7204,CANADA,"MbCeGY20kaKK3oalJD,OT",structions after the quietly ironic theodolite...,19672.2960
1,70,Customer#000000070,4867.52,32-828-107-2832,RUSSIA,mFowIuhnHjp2GjCiYYavkW kUwOjIaTCQ,fter the special asymptotes. ideas after the u...,40120.1103
2,97,Customer#000000097,2164.48,27-588-919-5638,PERU,"OApyejbhJG,0Iw3j rd1M","haggle slyly. bold, special ideas are blithely...",61459.7804
3,131,Customer#000000131,8595.53,21-840-210-3572,IRAQ,"jyN6lAjb1FtH10rMC,XzlWyCBrg75",jole special packages. furiously final depende...,43260.4800


In [131]:
# tpch - 11
var1 = 'PERU'
var2 = 0.0001

nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
partsupp = pandas.read_table(rf'{path}\partsupp.tbl', sep='|', index_col=False, header=None, names=pysdql.PARTSUPP_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)

sub_n = nation[nation['n_name'] == var1]

r1 = sub_n.merge(supplier, left_on='n_nationkey', right_on='s_nationkey')
r2 = r1.merge(partsupp, left_on='s_suppkey', right_on='ps_suppkey')

r2['agg_val'] = r2['ps_supplycost'] * r2['ps_availqty'] * var2

agg_val = r2['agg_val'].sum()

r = r2.groupby(['ps_partkey']).filter(lambda x: (x['ps_supplycost'] * x['ps_availqty']).sum() > agg_val)

r['value'] = r['ps_supplycost'] * r['ps_availqty']

r = r.groupby(['ps_partkey'], as_index=False).agg(value=('value', 'sum'))

r

,ps_partkey,value
0,1,1456050.96
1,3,2528031.80
2,4,1102037.60
3,5,487669.56
4,6,2142145.68
...,...,...
117,196,2429515.15
118,197,15327154.14
119,198,1403881.30
120,199,381417.48


In [132]:
# tpch - 12

import pandas
import numpy

var1 = 'FOB'
var2 = 'AIR'
var3 = '1995-01-01'
var4 = '1996-01-01'

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)

sub_l = lineitem[(lineitem['l_commitdate'] < lineitem['l_receiptdate'])
                 & (lineitem['l_shipdate'] < lineitem['l_commitdate'])
                 & (lineitem['l_receiptdate'] >= var3) & (lineitem['l_receiptdate'] < var4)]

sub_l = sub_l[lineitem['l_shipmode'].isin([var1, var2])]

r = orders.merge(sub_l, left_on='o_orderkey', right_on='l_orderkey')

r['high_line_priority'] = numpy.select(
[
    (r['o_orderpriority'] == '1-URGENT') | (r['o_orderpriority'] == '2-HIGH')
],
[
    1
],
default=0)

r['low_line_priority'] = numpy.select(
[
    (r['o_orderpriority'] != '1-URGENT') | (r['o_orderpriority'] != '2-HIGH')
],
[
    1
],
default=0)

r.groupby(['l_shipmode'], as_index=False).agg(high_line_count=('high_line_priority', 'sum'), low_line_count=('low_line_priority', 'sum'))

C:\Users\Y\AppData\Local\Temp\ipykernel_19592\2549720758.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sub_l = sub_l[lineitem['l_shipmode'].isin([var1, var2])]


,l_shipmode,high_line_count,low_line_count
0,AIR,8,13
1,FOB,6,12


In [133]:
# tpch - 14

var1 = '1995-03-01'
var2 = '1995-04-01'  # var1 + 1 month

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)

sub_l = lineitem[(lineitem['l_shipdate'] >= var1) & (lineitem['l_shipdate'] < var2)]
r = sub_l.merge(part, left_on='l_partkey', right_on='p_partkey')

r['tmp_val_1'] = numpy.select(
[
    r['p_type'].str.startswith('PROMO')
],
[
    r['l_extendedprice'] * (1 - r['l_discount'])
],
default=0)

r['tmp_val_2'] = r['l_extendedprice'] * (1 - r['l_discount'])

r['promo_revenue'] = 100 * r['tmp_val_1'].sum() / r['tmp_val_2'].sum()

r[['promo_revenue']].drop_duplicates()

,promo_revenue
0,9.627538


In [3]:
# tpch - 15

import pandas
import pysdql

var1 = '1996-02-01'
var2 = '1996-05-01'  # var1 + 3 month

path = r'T:\UG4-Proj\datasets'
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)

sub_l = lineitem[(lineitem['l_shipdate'] >= var1) & (lineitem['l_shipdate'] < var2)]

sub_l['tmp_val'] = sub_l['l_extendedprice'] * (1 - sub_l['l_discount'])

revenue0 = sub_l.groupby(['l_suppkey'], as_index=False).agg(total_revenue=('tmp_val', 'sum'))

revenue0['supplier_no'] = revenue0['l_suppkey']
revenue0 = revenue0[['supplier_no', 'total_revenue']]

max_revenue = revenue0['total_revenue'].max()

r = supplier.merge(revenue0, left_on='s_suppkey', right_on='supplier_no')

r = r[r['total_revenue'] == max_revenue]
r[['s_suppkey', 's_name', 's_address', 's_phone', 'total_revenue']]

C:\Users\Y\AppData\Local\Temp\ipykernel_24732\2580906340.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_l['tmp_val'] = sub_l['l_extendedprice'] * (1 - sub_l['l_discount'])


,s_suppkey,s_name,s_address,s_phone,total_revenue
8,9,Supplier#000000009,1KhUgZegwM3ua7dsYmekYBsK,20-403-398-8662,607520.0095


In [135]:
# tpch - 17

var1 = 'Brand#11'
var2 = 'WRAP CASE'

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)

part_agg = lineitem.groupby(['l_partkey'], as_index=False).agg(tmp_val=('l_quantity', 'mean'))

part_agg['avg_quantity'] = 0.2 * part_agg['tmp_val']
part_agg['agg_partkey'] = part_agg['l_partkey']

part_agg = part_agg[['avg_quantity', 'agg_partkey']]

sub_p = part[(part['p_brand'] == var1) & (part['p_container'] == var2)]

r = sub_p.merge(lineitem, left_on='p_partkey', right_on='l_partkey')
r = r.merge(part_agg, left_on='l_partkey', right_on='agg_partkey')

r = r[r['l_quantity'] < r['avg_quantity']]

r['avg_yearly'] = r['l_extendedprice'] / 7.0

r['avg_yearly'].sum()

1351.485714285714

In [136]:
# tpch - 18
var1 = 251

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)

aggr_l = lineitem.groupby(['l_orderkey'], as_index=False).filter(lambda x: x['l_quantity'].sum() > var1)

sub_o = orders[(orders['o_orderkey'].isin(aggr_l['l_orderkey']))]

r = customer.merge(sub_o, left_on='c_custkey', right_on='o_custkey')
r = r.merge(lineitem, left_on='o_orderkey', right_on='l_orderkey')

r = r.groupby(['c_name', 'c_custkey', 'o_orderkey', 'o_orderdate', 'o_totalprice'], as_index=False).agg({'l_quantity': 'sum'})

r

,c_name,c_custkey,o_orderkey,o_orderdate,o_totalprice,l_quantity
0,Customer#000000010,10,4421,1997-04-04,258779.02,255
1,Customer#000000068,68,2208,1995-05-01,245388.06,256
2,Customer#000000070,70,2567,1998-02-27,263411.29,266
3,Customer#000000082,82,3460,1995-10-03,245976.74,254


In [137]:
# tpch - 19
var1 = 'Brand#54'
var2 = 'Brand#22'
var3 = 'Brand#33'
var4 = 1
var4_1 = var4 + 10
var5 = 18
var5_1 = var5 + 10
var6 = 26
var6_1 = var6 + 10

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)

r = lineitem.merge(part, left_on='l_partkey', right_on='p_partkey')

r = r[((r['p_brand'] == var1)
       & (r['p_container'].isin(['SM CASE', 'SM BOX', 'SM PACK', 'SM PKG']))
       & (r['p_size'] >= 1) & (r['p_size'] <= 5) & (lineitem['l_quantity'] >= var4)
       & (r['l_quantity'] <= var4_1)
       & (r['l_shipmode'].isin(['AIR', 'AIR REG']))
       & (r['l_shipinstruct'] == 'DELIVER IN PERSON')
       )
      | ((r['p_brand'] == var2)
         & (r['p_container'].isin(['MED BAG', 'MED BOX', 'MED PKG', 'MED PACK']))
         & (r['p_size'] >= 1) & (r['p_size'] <= 10)
         & (r['l_quantity'] >= var5) & (r['l_quantity'] <= var5_1)
         & (r['l_shipmode'].isin(['AIR', 'AIR REG']))
         & (r['l_shipinstruct'] == 'DELIVER IN PERSON')
         )
      | (r['p_brand'] == var3)
         & (r['p_container'].isin(['LG CASE', 'LG BOX', 'LG PACK', 'LG PKG']))
         & (r['p_size'] >= 1) & (r['p_size'] <= 15)
         & (r['l_quantity'] >= var6)
         & (r['l_quantity'] <= var6_1)
         & (r['l_shipmode'].isin(['AIR', 'AIR REG']))
         & (r['l_shipinstruct'] == 'DELIVER IN PERSON')
         ]

r['tmp_val'] = r['l_extendedprice'] * (1 - r['l_discount'])

r['tmp_val'].sum()

57579.246

In [138]:
# tpch - 20

var1 = 'orange'
var2 = '1995-01-01'
var2_1 = '1996-01-01'  # var2 + 1 year
var3 = 'UNITED STATES'

lineitem = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=pysdql.LINEITEM_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
part = pandas.read_table(rf'{path}\part.tbl', sep='|', index_col=False, header=None, names=pysdql.PART_COLS)
partsupp = pandas.read_table(rf'{path}\partsupp.tbl', sep='|', index_col=False, header=None, names=pysdql.PARTSUPP_COLS)
supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)

sub_l = lineitem[(lineitem['l_shipdate'] >= var2) & (lineitem['l_shipdate'] < var2_1)]

agg_lineitem = sub_l.groupby(['l_partkey', 'l_suppkey'], as_index=False).agg(tmp_val=('l_quantity', 'sum'))

agg_lineitem['agg_partkey'] = agg_lineitem['l_partkey']
agg_lineitem['agg_suppkey'] = agg_lineitem['l_suppkey']
agg_lineitem['agg_quantity'] = agg_lineitem['tmp_val'] * 0.5

r = partsupp.merge(agg_lineitem, left_on='ps_partkey', right_on='agg_partkey')
r = r[(r['ps_suppkey'] == r['agg_suppkey']) & (r['ps_availqty'] > r['agg_quantity'])]

sub_p = part[part['p_name'].str.startswith(var1)]
r = r[(r['ps_partkey'].isin(sub_p['p_partkey']))]

sub_n = nation[(nation['n_name'] == var3)]
s = supplier.merge(sub_n, left_on='s_nationkey', right_on='n_nationkey')
s = s[(s['s_suppkey'].isin(r['ps_suppkey']))]
s[['s_name', 's_address']]

,s_name,s_address
0,Supplier#000000010,Saygah3gYWMp72i PY


In [188]:
# tpch - 21
var1 = 'UNITED KINGDOM'

supplier = pandas.read_table(rf'{path}\supplier.tbl', sep='|', index_col=False, header=None, names=pysdql.SUPPLIER_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)
nation = pandas.read_table(rf'{path}\nation.tbl', sep='|', index_col=False, header=None, names=pysdql.NATION_COLS)
l1_cols = ['l1_orderkey', 'l1_partkey', 'l1_suppkey', 'l1_linenumber', 'l1_quantity', 'l1_extendedprice',
           'l1_discount', 'l1_tax', 'l1_returnflag', 'l1_linestatus', 'l1_shipdate', 'l1_commitdate',
           'l1_receiptdate', 'l1_shipinstruct', 'l1_shipmode', 'l1_comment']
l1 = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=l1_cols)
l2_cols = ['l2_orderkey', 'l2_partkey', 'l2_suppkey', 'l2_linenumber', 'l2_quantity', 'l2_extendedprice',
           'l2_discount', 'l2_tax', 'l2_returnflag', 'l2_linestatus', 'l2_shipdate', 'l2_commitdate',
           'l2_receiptdate', 'l2_shipinstruct', 'l2_shipmode', 'l2_comment']
l2 = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=l2_cols)
l3_cols = ['l3_orderkey', 'l3_partkey', 'l3_suppkey', 'l3_linenumber', 'l3_quantity', 'l3_extendedprice',
           'l3_discount', 'l3_tax', 'l3_returnflag', 'l3_linestatus', 'l3_shipdate', 'l3_commitdate',
           'l3_receiptdate', 'l3_shipinstruct', 'l3_shipmode', 'l3_comment']
l3 = pandas.read_table(rf'{path}\lineitem.tbl', sep='|', index_col=False, header=None, names=l3_cols)

sub_n = nation[(nation['n_name'] == var1)]
join_ns = supplier.merge(sub_n, left_on='s_nationkey', right_on='n_nationkey')
sub_l1 = l1[(l1['l1_receiptdate'] > l1['l1_commitdate'])]
r = join_ns.merge(sub_l1, left_on='s_suppkey', right_on='l1_suppkey')

sub_o = orders[(orders['o_orderstatus'] == 'F')]
r = r.merge(sub_o, left_on='l1_orderkey', right_on='o_orderkey')

r = r.merge(l2, left_on='l1_orderkey', right_on='l2_orderkey')

r['exists_l2'] = numpy.select(
[
    (r['l1_suppkey'] != r['l2_suppkey'])
],
[
    1
],
default=0)

main_cols = pysdql.SUPPLIER_COLS + pysdql.ORDERS_COLS + pysdql.NATION_COLS + l1_cols

r = r.groupby(main_cols, as_index=False).agg(exists_l2=('exists_l2', 'sum'))
r = r[r['exists_l2'] > 0]

r = r.merge(l3, left_on='l1_orderkey', right_on='l3_orderkey')
r['exists_l3'] = numpy.select(
[
    (r['l1_suppkey'] != r['l3_suppkey'])
    & (r['l3_receiptdate'] > r['l3_commitdate'])
],
[
    1
],
default=0)
r = r.groupby(main_cols, as_index=False).agg(exists_l3=('exists_l3', 'sum'))
r = r[r['exists_l3'] == 0]

r.groupby(['s_name'], as_index=False).agg(numwait=('l1_orderkey', 'count'))

,s_name,numwait
0,Supplier#000000007,17


In [203]:
# tpch - 22

var1 = ('16', '12', '18', '14', '30', '27', '25')

customer = pandas.read_table(rf'{path}\customer.tbl', sep='|', index_col=False, header=None, names=pysdql.CUSTOMER_COLS)
orders = pandas.read_table(rf'{path}\orders.tbl', sep='|', index_col=False, header=None, names=pysdql.ORDERS_COLS)

sub_c = customer[(customer['c_acctbal'] > 0.00)
                 & (customer['c_phone'].str.slice(0, 2).isin(var1))]

avg_acctbal = sub_c['c_acctbal'].mean()

r = customer[customer['c_phone'].str.slice(0, 2).isin(var1)]

r = r[r['c_acctbal'] > avg_acctbal]

r = r.merge(orders, how='cross')
r['exists'] = numpy.select(
[
    (r['o_custkey'] == r['c_custkey'])
],
[
    1
],
default=0)
r = r.groupby(pysdql.CUSTOMER_COLS, as_index=False).agg(exists=('exists', 'sum'))
r = r[r['exists'] == 0]

r['cntrycode'] = customer['c_phone'].str.slice(0, 2)

custsale = r[['cntrycode', 'c_acctbal']]

custsale.groupby(['cntrycode'], as_index=False).agg(numcust=('cntrycode', 'count'), totacctbal=('c_acctbal', 'sum'))

,cntrycode,numcust,totacctbal
0,11,1,8324.07
1,14,1,5494.43
2,23,1,6323.92
3,25,1,7638.57
4,30,1,6264.31
